# Overview of Methods

For each ward:

1. Read in the centers for each image.
2. Adjust the centers down.
3. Extract two images of size 400x425 from each adjusted center. One plain image and one with ward ovelayed in pure black.
4. Write each image to the directory /images/ward_k/ij_with_ward.png where k is the ward number and ij are the row and column positions, respectively, of the image in the grid of images from the specific ward (e.g. top left = 00).
5. Use the image with the ward overlayed to isolate the pixels that are outside of the ward. Set those pixels to black in the image without the ward in order to isolate the section of the image that contains the ward.
6. Remove the Google watermarks (i.e. bottom 25 pixels).
7. Write the edited image to the directory /images/ward_k/edited/ij.png

# Libraries Used

In [3]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import json
import requests
import pandas as pd
import os
import ast

# Constants

### Conversion of lat/lng to pixels

In order to sample without overlap from the grid of images within the ward, we need to obtain the length of the sampled images in the y- and the x-direction.

In [4]:
## Radius of squares in grid, figured out by mixture of observation and calculation
Y_RAD = 0.00385
X_RAD = 0.0043
N_WARDS = 130

### Adjustment to crop out watermarks

Although we need images of dimension 400x400, we are sampling images of size 400x425 (more pixels in the y-direction) and cropping the bottom 25px in order to remove the Google watermarks - resulting in a 400x400 image. However, to do this, we need to shift the center of our sampled images slighly lower, as we are removing pixels from only the bottom of the image, even though there are $\approx 12.5$ pixels at the top of the image that are also being added. Thus, we need to shift the center down "12.5 pixels". Converting this to lat/lng values, we get that:
$200$px $\approx 0.00385^{\circ}$. So it follows that $12.5$px $\approx 0.00385^{\circ}/16$

<img src="../images_for_demo/Grid_img_without_adjustment.PNG" width="300">
<img src="../images_for_demo/Grid_img_with_adjustment.PNG" width="300">

In [5]:
DELTA_Y = Y_RAD/16  # adjustment to crop out watermarks

In [6]:
# get the Google Static Maps API key
with open("../google_api_key.txt", "r") as file:
    KEY = file.readline()

# Extract Images for each Ward

In [14]:
# read in ward image centers
df_ward_centers = pd.read_csv("../data/ward_image_centers.csv") 
print(df_ward_centers.dtypes)
df_ward_centers.head()

ward_no.        int64
img_centers    object
dtype: object


,ward_no.,img_centers
0,1,"[[(-26.483020238999927, 27.828767997000057), (..."
1,2,"[[(-26.496054001999937, 27.86212946200004), (-..."
2,3,"[[(-26.47171999699998, 27.854994917000056), (-..."
3,4,"[[(-26.448481038999944, 27.84236260700004), (-..."
4,5,"[[(-26.424091998999927, 27.763540001000024), (..."


In [33]:
def download_img(center_str, path):
    request = "https://maps.googleapis.com/maps/api/staticmap?center=" + center_str + "&zoom=16&size=400x425&maptype=satellite&format=png&path=fillcolor:0x000000FF|color:0x000000FF|weight:1" + path + "&key=" + KEY
    # HTTP requests
    return(requests.get(request))

In [34]:
def get_imgs(center_str, ward_num, pos, path):
    
    # extrecting two images for each center
    response_with_ward = download_img(center_str, path)     # image with ward overlayed in black
    response_no_ward = download_img(center_str, "")         # plain image
    
    if not os.path.exists('../images/ward_' + str(ward_num)):
        os.makedirs('../images/ward_' + str(ward_num))
    
    # storing the responses in a file (images)
    with open('../images/ward_' + str(ward_num) + "/" + str(pos[0]) + str(pos[1]) + "_with_ward" +'.png', 'wb') as file:
       # writing data into the file
       file.write(response_with_ward.content)

    with open('../images/ward_' + str(ward_num) + "/" + str(pos[0]) + str(pos[1]) +'.png', 'wb') as file:
       # writing data into the file
       file.write(response_no_ward.content)

In [90]:
# function for converting coordinates to format for Google Static Maps API
def center_to_string(center):
    return str(center[0]) + "," + str(center[1])

In [27]:
# Dowload images from Google Static Maps API for each ward
# Saved in file ./images/ward_i
# Two imgs from same location: 
## (row,col).png - plain img
## (row,col)_with_ward.png - img with ward overlayed
# (row,col) are matrix coordinates
# use ast.literal_eval: '[1,2]' -> [1,2]

ward_num = 1    # could use apply or map here
for i, cols in enumerate(ast.literal_eval(df_ward_centers["img_centers"][ward_num-1])):
    coords = get_ward_shape(shape_file_dict, ward_num-1)
    path = coords_to_path(coords)
    for j, row_center in enumerate(cols):
        center_shifted = (row_center[0] - DELTA_Y, row_center[1])              # shifting the center down by DELTA_Y
#         get_imgs(center_to_string(center_shifted), ward_num, (i,j), path)    # uncomment to download images
    

# Edit Plain Images to only Contain Ward Area and Save

In [96]:
# For saving the edited image to ./images/ward_i/edited
def save_edited_img(ward_num, f_no_ward, img):
    # create directory if it doesn't exist
    if not os.path.exists('../images/ward_' + str(ward_num) + '/edited'):
        os.makedirs('../images/ward_' + str(ward_num)+ '/edited')
    
    # writing the image to directory
    cv2.imwrite('../images/ward_' + str(ward_num)+ '/edited/' + f_no_ward, img)

In [99]:
# For setting the area of the img outside of the ward to black
# Edited image saved to ./images/ward_i/edited
def isolate_ward_in_img(ward_num, f_with_ward, f_no_ward):
    img_with_ward = cv2.imread('../images/ward_' + str(ward_num) + '/' + f_with_ward)
    img_no_ward = cv2.imread('../images/ward_' + str(ward_num)+ '/' + f_no_ward)
    
    img_with_ward_cropped = img_with_ward[0:400,:,:]      # cropping out the watermarks
    img_no_ward_cropped = img_no_ward[0:400,:,:]          # cropping out the watermarks
    
    # selecting all non-black pixels from image with ward overlayed (want to remove these from plain image)
    pix_out_ward = np.where(np.all(img_with_ward_cropped != [0,0,0], axis=-1))
    
    # setting all non-black pixels from image without ward overlayed to black in plain image
    img_no_ward_cropped[pix_out_ward] = [0,0,0]
    return img_no_ward_cropped

In [100]:
# looping through the directories for each ward and editing the images then saving

ward_num = 1
directory = os.fsencode("../images/ward_" + str(ward_num))
i = 0
f_with_ward = f_no_ward = ''
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".png"):
        if i % 2 == 0:
            f_no_ward = filename
        else:
            f_with_ward = filename
            img_no_ward_cropped = isolate_ward_in_img(ward_num, f_with_ward, f_no_ward)
            save_edited_img(ward_num, f_no_ward, img_no_ward_cropped)
        i += 1

In [8]:
def plot_fig(img):
    img_matplot = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(5,5))
    plt.imshow(img_matplot)